In [11]:
import pandas as pd
import os

In [19]:
# data processed to sorted by years
path="./datas_processed/"
files = os.listdir(path)
csv_files = [file for file in files if file.endswith('.csv')]
yr=range(2004,2023)

for i in yr:
    data=pd.DataFrame()
    for file in csv_files:
        tmp=pd.read_csv(path+file)
        data[file[:-4]]=tmp[f"{i}"]
    tmp=pd.read_csv("./data/GDP.csv")
    data.index=tmp["地区"]
    data.to_csv(f"./sorted/{i}.csv")

In [28]:
# sorted by years to sorted by region
path="./sorted/"
yr=range(2004,2023)
tmp=pd.read_csv("./data/GDP.csv")
location=tmp["地区"].values
for loct in location:
    data=pd.DataFrame()
    for i in yr:
        tmp=pd.read_csv(path+f"{i}.csv")
        addrow=tmp.loc[tmp["地区"]==loct]
        data=pd.concat([data,addrow])
    data["Year"]=range(2004,2023)
    data.set_index("Year",inplace=True)
    data.drop("地区",axis=1, inplace=True)
    data.to_csv(f"./region/{loct}.csv")

In [14]:
# import os
# import pandas as pd

# path="./region/"

# files=os.listdir(path)

# ret=pd.DataFrame()

# for file in files:
#     tmp=pd.read_csv(path+file)
#     ret=pd.concat([ret,tmp])

# ret=ret.set_index("Year")

In [15]:
from numpy import poly1d,polyfit

def extend(df,target):
    df.dropna(inplace=True)
    f = poly1d(polyfit(df["Year"],df[target],1))
    year=range(2004,2023)
    ret=pd.DataFrame()
    ret["Year"]=year
    t_v=[]
    for i in year:
        t_v.append(df.loc[df['Year'] == i, target].values[0] if sum(df['Year'] == i) > 0 else f(i))
    ret[target]=t_v
    return ret

In [25]:

stable=pd.read_csv("./data/stable_index.csv")
stable=stable[["地区","Year","财政收入稳健指数"]]
epu=pd.read_csv("./data/epu.csv")
epu.drop("province_code",axis=1, inplace=True)
provs=stable["地区"].unique()
for prov in provs:
    tmp=stable[stable["地区"]==prov].copy()
    tmp2=epu[epu["地区"]==prov].copy()
    tmp["Year"]=tmp["Year"].round(0).astype(int)
    tmp.drop("地区",axis=1,inplace=True)
    tmp=extend(tmp,"财政收入稳健指数")
    tmp2.drop("地区",axis=1,inplace=True)
    tmp2=extend(tmp2,"EPU")
    tmp=pd.merge(tmp,tmp2,on='Year',how='outer')
    tmp.set_index("Year",inplace=True)
    tmp.sort_index(ascending=True,inplace=True)
    # tmp.drop(range(2000,2008),inplace=True)
    tmp.to_csv(f"./tmp/{prov}省.csv")

In [26]:
path1="./tmp/"
path2="./region/"
names1=os.listdir(path1)
names2=os.listdir(path2)

matching_files = [file1 for file1 in names1 for file2 in names2 if file1[:2] == file2[:2]]

for file1 in names1:
    for file2 in names2:
        if file2[:2]==file1[:2]:
            match = file2
            break
    os.rename(path1+file1,path1+match)
    # df_region=pd.read_csv(path2+match)
    # df_tmp=pd.read_csv(path1+match)

In [29]:
for file1 in names1:
    for file2 in names2:
        if file2[:2]==file1[:2]:
            match = file2
            break
    df_region=pd.read_csv(path2+match)
    df_tmp=pd.read_csv(path1+match)
    final_ret=df_region
    final_ret["财政收入稳健指数"]=df_tmp["财政收入稳健指数"]
    final_ret["EPU"]=df_tmp["EPU"]
    final_ret.set_index('Year',inplace=True)
    final_ret.sort_index(inplace=True)
    final_ret.to_csv(path2+match)